This section aims to try different approaches. Some of the project's choices might be hindering the model's performance such as brand-model grouped imputation.

#

In [39]:
# Locate project root dir, enable package imports from src/
import sys
from pathlib import Path

PROJ_ROOT = Path().resolve().parents[1]
sys.path.append(str(PROJ_ROOT / "src"))
print(PROJ_ROOT)

/Users/alexandre/Documents/GitHub/price_predictor


In [40]:
import os
import pandas as pd
import numpy as np
import joblib  
import json

# Load standardized paths, ml utilities and custom transformer
from project_utils.paths import (
    CLEANED_DATA_DIR,
    TEST_PREDICTIONS_DIR,
)
import project_utils.ml_utils as ml_utils
import project_utils.custom_transformers as custom_transformers

# Data pre-processing
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

# Model and random-search and cross-validation
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import (
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
    ExtraTreesRegressor,
)

from sklearn.model_selection import train_test_split

In [41]:
df_train = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_train_data.csv"), index_col="carID"
)
X_test = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_test_data.csv"), index_col="carID"
)

X = df_train.drop("price", axis=1).copy()
y = df_train["price"].copy()

# Performing the holdout methods
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

# Experimenting new pipeline

In this notebook, the analysis focuses exclusively on decision tree–based models, which are expected to provide strong predictive performance for this task.

In [42]:
pipeline = joblib.load("best_sklearn_preprocessor.joblib")
model = joblib.load("best_sklearn_model.joblib")

In [43]:
model

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('column_transformer', ...), ('scaler', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'

+ IterativeImputer - Iterative imputation was employed as a strategy for handling missing numerical values by modelling each feature with missing entries as a function of the remaining features. Given the presence of multiple multivariate relationships within the dataset, this approach has the potential to yield more accurate imputations than univariate methods.

+ SimpleImputer - For categorical variables, missing values were imputed using a simple imputation strategy based on the most frequent category per feature. This method preserves the modal structure of categorical variables while avoiding the introduction of artificial categories.

+ StandardScaler - As several highly skewed numerical features were previously log-transformed, a z-score standardisation approach was adopted for feature scaling. This method centres features to zero mean and unit variance, facilitating stable optimisation and comparability across numerical variables.

It should also be noted that, within this pipeline, categorical features were not scaled. This design choice is justified by the focus of this notebook on tree-based models, which are invariant to feature scaling. In contrast, scaling was applied after target encoding in earlier pipelines, as failing to scale categorical features transformed into continuous variables can adversely affect the performance of scale-sensitive models, such as linear and instance-based regressors.

In [44]:
# --- Defining numerical and categorical feature columns --- #
numeric_cols = X_train.select_dtypes(exclude="object").columns.to_list()
categorical_cols = X_train.select_dtypes(include="object").columns.to_list()

Features identified for removal through filter-based methods were excluded from subsequent modelling stages, with the exception of those discarded by the Elastic Net regressor. These features were retained, as Elastic Net’s linear and additive nature may suppress variables that are weak in isolation but informative when combined or exploited by non-linear models. Consequently, preserving these features allows downstream models to capture complex relationships that Elastic Net may fail to represent.

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler, TargetEncoder

pipeline = Pipeline(
    steps=[
        (
            "column_transformer",
            ColumnTransformer(
                transformers=[
                    # --- Numeric pipeline ---
                    (
                        "num",
                        Pipeline(
                            steps=[
                                ("imputer", IterativeImputer(estimator=None, n_nearest_features=5, random_state=0)),
                                ("scaler", StandardScaler()),
                            ]
                        ),
                        numeric_cols,
                    ),
                    # --- Categorical pipeline ---
                    (
                        "cat",
                        Pipeline(
                            steps=[
                                ("imputer", SimpleImputer(strategy="most_frequent")),
                                (
                                    "encoder",
                                    TargetEncoder(
                                        categories="auto",
                                        target_type="continuous",
                                        smooth="auto",
                                    ),
                                ),
                            ]
                        ),
                        categorical_cols,
                    ),
                ]
            ),
        )
    ]
)


In [46]:
# Import JSON file with best_params selected from the RandomSearch
RANDOM_SEARCH = os.path.join(PROJ_ROOT, "src", "models", "random_search")
with open(os.path.join(RANDOM_SEARCH, "best_hgbr_params.json"), "r") as f:
    best_params = json.load(f)

clean_params = {
    k.replace("regressor__regressor__", ""): v for k, v in best_params.items()
}

# --- HGBR with log-transform target --- #
log_tree_regressor = TransformedTargetRegressor(
    regressor=HistGradientBoostingRegressor(**clean_params),
    func=np.log1p,
    inverse_func=np.expm1,
)

model = Pipeline(
    [
        ("preprocessor", pipeline),
        ("regressor", log_tree_regressor),
    ]
)
# # --- K-Fold CV ---#
cv = KFold(n_splits=10, shuffle=True, random_state=0)

scores = cross_val_score(
    model, X_train, y_train, scoring="neg_mean_absolute_error", cv=cv, n_jobs=-1
)

hgbr_model = pipeline.fit(X, y)
hgbr_model

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('column_transformer', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains spa

In [47]:
# Compute evaluation metrics
model.fit(X, y)
y_pred = model.predict(X)
mae = ml_utils.linear_evaluation_metrics(y, y_pred, verbose=False)[1]
print("Training MAE:", round(mae, 2))
mae_scores = -scores
print("CV MAE mean:", round(mae_scores.mean(), 2))
print("CV MAE std :", round(mae_scores.std(), 2))

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

Training MAE: 1217.36
CV MAE mean: 1454.1
CV MAE std : 24.74


In [48]:
model

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('column_transformer', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} 

In [49]:
# --- ETR with log-transform target --- #
extr = ExtraTreesRegressor(
    criterion="squared_error",
    random_state=0,
    bootstrap=True,
    n_jobs=-1,
)

log_extr = TransformedTargetRegressor(
    regressor=extr,
    func=np.log1p,
    inverse_func=np.expm1,
)

pipeline = Pipeline(
    [
        ("preprocessor", pipeline),
        ("regressor", log_extr),
    ]
)

# --- K-Fold CV ---#
cv = KFold(n_splits=10, shuffle=True, random_state=0)

scores = cross_val_score(
    pipeline, X_train, y_train, scoring="neg_mean_absolute_error", cv=cv, n_jobs=-1
)
etr_model = pipeline.fit(X_train, y_train)
etr_model

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('column_transformer', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} 

In [50]:
# Compute evaluation metrics
y_pred = pipeline.predict(X)
mae = ml_utils.linear_evaluation_metrics(y, y_pred, verbose=False)[1]
print("Training MAE:", round(mae, 2))
mae_scores = -scores
print("CV MAE mean:", round(mae_scores.mean(), 2))
print("CV MAE std :", round(mae_scores.std(), 2))

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

Training MAE: 1025.11
CV MAE mean: 1481.76
CV MAE std : 20.72


In [51]:
# Compute evaluation metrics
y_pred = pipeline.predict(X)
mae = ml_utils.linear_evaluation_metrics(y, y_pred, verbose=False)[1]
print("Training MAE:", round(mae, 2))
mae_scores = -scores
print("CV MAE mean:", round(mae_scores.mean(), 2))
print("CV MAE std :", round(mae_scores.std(), 2))


/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

Training MAE: 1025.11
CV MAE mean: 1481.76
CV MAE std : 20.72


In [ ]:
# --- Scikit-learn: ensemble & models ---
from sklearn.ensemble import (
    BaggingRegressor,
    StackingRegressor,
)
from sklearn.linear_model import RidgeCV



# Decision tree models
hgbr_model = hgbr_model
etr_model = etr_model
# KNN baggging
bagging_regressor = Pipeline(
    [
        ("preprocessor", pipeline),
        ("regressor",log_extr = TransformedTargetRegressor(
    regressor=BaggingRegressor(),
    func=np.log1p,
    inverse_func=np.expm1,
) ),
    ]
)
estimators = [
    ("hgbr", hgbr_model),
    ("etr", etr_model),
    ("bg", bagging_regressor),
]

# --- RidgeCV meta-learner ---
ridge_meta = RidgeCV(alphas=np.logspace(-4, 4, 20), cv=10)

stacking_ensemble = StackingRegressor(
    estimators=estimators, final_estimator=ridge_meta, cv=10, n_jobs=-1
)

reg = TransformedTargetRegressor(
    regressor=stacking_ensemble, func=np.log1p, inverse_func=np.expm1
)

reg.fit(X_train, y_train)


python(63073) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63074) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63075) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63076) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63077) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63078) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63079) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63080) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63081) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63082) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


ValueError: could not convert string to float: 'manual'